In [82]:
import re
import time

import pandas as pd
import string
#os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing')
directory_path = 'STAR_eval/Charades'
from gpt_ask import run_gpt

In [83]:
main_ds = pd.read_json('STAR_eval/STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

In [84]:
df = pd.read_csv('STAR_eval/new_all_cured_old.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna()
df = df.drop_duplicates()

In [85]:
df['OIC_answer_without_CUID'] =  df['OIC_context'].apply(lambda x: re.sub('_[a-zA-Z0-9]*', '', x))
df['OIC_answer_without_CUID'][0]

'The scene opens:\nFrom now on man wearing shirt\nFrom now on man wearing hat\nFrom now on man wearing pant\nFrom now on man wearing glove\nFrom now on man has head\nFrom now on man has arm\nFrom now on man has ear\nFrom now on man wearing glass\nFrom now on shirt on man\nFrom now on hat on man\nFrom now on pant on man\nman has ear is not longer actual.\npant on man is not longer actual.\nAfter 1 seconds:\nAfter 2 seconds:\nFrom now on man has ear\nFrom now on hat on head\nFrom now on pant on man\nAfter 3 seconds:\nman wearing shirt is not longer actual.\nman has ear is not longer actual.\nshirt on man is not longer actual.\nhat on man is not longer actual.\nhat on head is not longer actual.\npant on man is not longer actual.\nAfter 4 seconds:\nFrom now on man has hair\nFrom now on man sitting on toilet\nFrom now on man wearing tie\nFrom now on man wearing jean\nFrom now on hair of man\nFrom now on ear of man\nFrom now on tie on man\nFrom now on hand of man\nFrom now on person wearing 

In [86]:
class Substitutable(str):
  def __new__(cls, *args, **kwargs):
    newobj = str.__new__(cls, *args, **kwargs)
    newobj.sub = lambda fro,to: Substitutable(re.sub(fro, to, newobj))
    return newobj

In [87]:
df['OIC_answer_without_CUID_temp'] = df['OIC_answer_without_CUID']
df['OIC_answer_without_CUID_temp'] = df['OIC_answer_without_CUID_temp'].apply(lambda x: Substitutable(x).sub('\n', ' ').sub('The scene opens:', ' ').sub('From now on', ' ').sub('is not longer actual.',' ').sub('\nAfter [0-9]* seconds:', ' '))

df['OIC_answer_without_CUID_temp']

0           man wearing shirt   man wearing hat   man ...
1           glove on hand   man wearing pant   man wea...
2           man wearing shirt   man has hair   man wea...
3           woman wearing shirt   woman has hair   wom...
4           man wearing shirt   man wearing pant   man...
                              ...                        
1569        handle on door   door has handle After 1 s...
1570        woman has hair   woman wearing shirt   wom...
1571        man wearing shirt   man wearing pant   man...
1572        woman wearing glass   woman in shirt   wom...
1573        man has hair   man wearing shirt   man sit...
Name: OIC_answer_without_CUID_temp, Length: 507, dtype: object

In [88]:
df.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match,OIC_answer_without_CUID,OIC_answer_without_CUID_temp
0,Interaction_T1_76,Which object was thrown by the person?,XO8NL,3.6,12.5,The pillow.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on man_f89c wearing...,3.0,Which object was thrown by the person? Guess t...,Correct,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man wearing hat man ...
1,Interaction_T1_78,Which object was opened by the person?,OY3LS,2.5,10.1,The laptop.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",The scene opens:\nFrom now on glove_beed on ha...,3.0,Which object was opened by the person? Guess t...,Wrong,The scene opens:\nFrom now on glove on hand\nF...,glove on hand man wearing pant man wea...
2,Interaction_T1_101,Which object was put down by the person?,RG0KS,17.4,27.0,The box.,"[{'choice_id': 0, 'choice': 'The cup/glass/bot...",The scene opens:\nFrom now on man_3171 wearing...,4.0,Which object was put down by the person? Guess...,Wrong,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man has hair man wea...
3,Interaction_T1_137,Which object was closed by the person?,NUKJ0,19.4,26.6,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",The scene opens:\nFrom now on woman_24bd weari...,3.0,Which object was closed by the person? Guess t...,Wrong,The scene opens:\nFrom now on woman wearing sh...,woman wearing shirt woman has hair wom...
4,Interaction_T1_171,Which object was put down by the person?,LBRYS,8.9,13.4,The shoe.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",The scene opens:\nFrom now on man_753c wearing...,2.0,Which object was put down by the person? Guess...,Wrong,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man wearing pant man...


In [89]:
from time import sleep
import time
import numpy as np
q_ids = np.unique(df['question_id'])
print(len(q_ids))
for q in q_ids:
  que = df.query("question_id == '"+q+"'")
  question = que['question'].values[0]
  answer = que['answer'].values[0]
  choices = dict()
  choice_string = ''
  options = main.query("question_id == '"+q+"'")["choices"].values[0]
  for choice in options:
        choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)).strip()})
        choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))
  
  que_df = df.query("question_id == '"+q+"'")
  prompt = que_df['OIC_answer_without_CUID'].values[0] # update column name to OIC_answer_without_CUID_temp
  formatted_question = question+' Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. If not found in the given options reply 0 not any other statements.'
  response = run_gpt(prompt, formatted_question)
  while True:
    if len(response)>1:
        response = run_gpt(prompt, formatted_question) 
    else:
      response = int(response)
      break
  if int(response) > 0:
    OIC_answer = response
    if choices[int(OIC_answer)-1] in answer:
      df.loc[df['question_id'] == q, 'Wo_CUID_Match'] = 'Correct' # update column name to Wo_CUID_temp
      print(df.query("question_id == '"+q+"'")['Wo_CUID_Match'].values[0])
    else:
      df.loc[df['question_id'] == q, 'Wo_CUID_Match'] = 'Wrong' # update column name to Wo_CUID_temp
      print(df.query("question_id == '"+q+"'")['Wo_CUID_Match'].values[0])
  else:
    df.loc[df['question_id'] == q, 'Wo_CUID_Match'] = 'Wrong'
  df.to_csv('STAR_eval/new_all_cured.csv') # update column name to without_cuid_temp  
  #df.drop(df[df['Wo_CUID_Match'].isnull().values.any()].index, inplace = True)
  #sleep(100)
main.head()
main.name = 'all data'

507


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}